In [1]:
import torch
import numpy as np
import cv2
import easyocr
from pathlib import Path

import pathlib
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath

# Assuming YOLOv5's utils and models are in the Python path
from models.experimental import attempt_load
from utils.general import non_max_suppression, scale_boxes
from utils.augmentations import letterbox

# Function to load the YOLO model and detect objects
def load_model(weights_path):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = attempt_load(weights_path, device=device)
    stride = int(model.stride.max())
    return model, stride, device

def detect_and_crop_objects(model, stride, device, image_path, conf_thres=0.25, iou_thres=0.45, imgsz=640):
    img0 = cv2.imread(image_path)
    assert img0 is not None, f'Image Not Found {image_path}'

    img = letterbox(img0, imgsz, stride=stride)[0]
    img = img[:, :, ::-1].transpose(2, 0, 1)
    img = np.ascontiguousarray(img)
    img = torch.from_numpy(img).to(device)
    img = img.float() / 255.0  # Normalize
    if img.ndimension() == 3:
        img = img.unsqueeze(0)

    pred = model(img, augment=False)[0]
    pred = non_max_suppression(pred, conf_thres, iou_thres)

    cropped_images = []
    for i, det in enumerate(pred):
        if len(det):
            det[:, :4] = scale_boxes(img.shape[2:], det[:, :4], img0.shape).round()
            for *xyxy, conf, cls in reversed(det):
                x1, y1, x2, y2 = map(int, xyxy)
                cropped_img = img0[y1:y2, x1:x2]
                cropped_images.append(cropped_img)
    return cropped_images

def apply_mask_for_ocr(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    normalized_gray = cv2.normalize(gray, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    _, img_plate_bw = cv2.threshold(normalized_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    img_plate_bw = cv2.morphologyEx(img_plate_bw, cv2.MORPH_OPEN, kernel, iterations=2)

    # Find contours and filter them
    contours, _ = cv2.findContours(img_plate_bw, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    filtered_contours = []
    for contour in contours:
        area = cv2.contourArea(contour)
        if area < 100: continue
        x, y, w, h = cv2.boundingRect(contour)
        aspect_ratio = float(w) / h
        if 0.2 < aspect_ratio < 1.0 and gray.shape[0] * 0.3 < h < gray.shape[0] * 0.8 and gray.shape[1] * 0.02 < w < gray.shape[1] * 0.15:
            filtered_contours.append(contour)

    # Create a blank mask that matches the image size
    mask = np.zeros_like(gray)  # Use gray instead of img

    # Draw the filtered contours on the mask with white color and full opacity
    cv2.drawContours(mask, filtered_contours, -1, (255, 255, 255), thickness=cv2.FILLED)

    # Apply the mask to the original image
    img_masked = cv2.bitwise_and(image, image, mask=mask)  # Use image instead of img
    
    return img_masked

def extract_text_from_image_with_masking(cropped_images):
    reader = easyocr.Reader(['en'])
    combined_text = ""
    for img in cropped_images:
        masked_img = apply_mask_for_ocr(img)
        results = reader.readtext(masked_img)
        for (_, text, _) in results:
            combined_text += text + " "
    return combined_text.strip()

# Then, in your process_image function or wherever appropriate, call this new function instead:
def process_image_with_masking(image_path, weights_path):
    model, stride, device = load_model(weights_path)
    cropped_images = detect_and_crop_objects(model, stride, device, image_path)
    detected_text = extract_text_from_image_with_masking(cropped_images)
    return detected_text

# Example usage
weights_path = './runs/train/exp3/weights/best.pt'  # Update this path
image_path = '../4224933456.jpg'  # Update this path
detected_text = process_image_with_masking(image_path, weights_path)
print(f"Detected Text: {detected_text}")


Fusing layers... 
Model summary: 157 layers, 7047883 parameters, 0 gradients, 15.9 GFLOPs


Detected Text: B  8 LONG


In [1]:
import torch
import numpy as np
import cv2
import easyocr
from models.experimental import attempt_load
from utils.general import non_max_suppression, scale_boxes
from utils.augmentations import letterbox
import argparse

import pathlib
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath

def load_model(weights_path):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = attempt_load(weights_path, device=device)
    stride = int(model.stride.max())
    return model, stride, device

def detect_and_crop_objects(model, stride, device, image_path, conf_thres=0.25, iou_thres=0.45, imgsz=640):
    img0 = cv2.imread(image_path)
    assert img0 is not None, f'Image Not Found {image_path}'

    img = letterbox(img0, imgsz, stride=stride)[0]
    img = img[:, :, ::-1].transpose(2, 0, 1)
    img = np.ascontiguousarray(img)
    img = torch.from_numpy(img).to(device)
    img = img.float() / 255.0  # Normalize
    if img.ndimension() == 3:
        img = img.unsqueeze(0)

    pred = model(img, augment=False)[0]
    pred = non_max_suppression(pred, conf_thres, iou_thres)

    cropped_images = []
    for i, det in enumerate(pred):
        if len(det):
            det[:, :4] = scale_boxes(img.shape[2:], det[:, :4], img0.shape).round()
            for *xyxy, conf, cls in reversed(det):
                x1, y1, x2, y2 = map(int, xyxy)
                cropped_img = img0[y1:y2, x1:x2]
                cropped_images.append(cropped_img)
    return cropped_images

def apply_mask_for_ocr(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    normalized_gray = cv2.normalize(gray, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    _, img_plate_bw = cv2.threshold(normalized_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    img_plate_bw = cv2.morphologyEx(img_plate_bw, cv2.MORPH_OPEN, kernel, iterations=2)

    contours, _ = cv2.findContours(img_plate_bw, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    filtered_contours = []
    for contour in contours:
        area = cv2.contourArea(contour)
        if area < 100: continue
        x, y, w, h = cv2.boundingRect(contour)
        aspect_ratio = float(w) / h
        if 0.2 < aspect_ratio < 1.0 and gray.shape[0] * 0.3 < h < gray.shape[0] * 0.8 and gray.shape[1] * 0.02 < w < gray.shape[1] * 0.15:
            filtered_contours.append(contour)

    mask = np.zeros_like(gray)
    cv2.drawContours(mask, filtered_contours, -1, (255, 255, 255), thickness=cv2.FILLED)
    img_masked = cv2.bitwise_and(image, image, mask=mask)
    return img_masked

def extract_text_from_image_with_masking(cropped_images):
    reader = easyocr.Reader(['en'], gpu=True if torch.cuda.is_available() else False)
    combined_text = ""
    for img in cropped_images:
        masked_img = apply_mask_for_ocr(img)
        results = reader.readtext(masked_img)
        for (_, text, _) in results:
            combined_text += text + " "
    return combined_text.strip()

def process_image_with_masking(image_path, weights_path):
    model, stride, device = load_model(weights_path)
    cropped_images = detect_and_crop_objects(model, stride, device, image_path)
    detected_text = extract_text_from_image_with_masking(cropped_images)
    return detected_text

def parse_args():
    parser = argparse.ArgumentParser(description="Detect and extract text from images.")
    parser.add_argument("--image_path", type=str, required=True, help="Path to the input image")
    parser.add_argument("--weights_path", type=str, required=True, help="Path to the YOLO model weights")
    return parser.parse_args()

def main():
    args = parse_args()
    detected_text = process_image_with_masking(args.image_path, args.weights_path)
    print(f"Detected Text: {detected_text}")

if __name__ == "__main__":
    main()


usage: ipykernel_launcher.py [-h] --image_path IMAGE_PATH --weights_path
                             WEIGHTS_PATH
ipykernel_launcher.py: error: the following arguments are required: --image_path, --weights_path


SystemExit: 2

C:\Users\Faturrahman\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
import torch
import torch.onnx

import pathlib
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath

# Path to your YOLOv5 model file
model_path = './runs/train/exp3/weights/best.pt'

# Load the model
model = torch.load(model_path)['model'].float()  # Casting to float to ensure compatibility
model.eval()

# Dummy input to the model
x = torch.randn(1, 3, 224, 224)  # Adjust the size according to your model's requirement

# Export the model
torch.onnx.export(model,               # model being run
                  x,                   # model input (or a tuple for multiple inputs)
                  "model.onnx",        # where to save the model
                  export_params=True,  # store the trained parameter weights inside the model file
                  opset_version=10,    # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})
import onnx
from onnx_tf.backend import prepare

# Load the ONNX file
model = onnx.load('model.onnx')

# Import the ONNX model to TensorFlow
tf_rep = prepare(model)

# Export the TensorFlow model
tf_rep.export_graph('model_tf')
import tensorflow as tf

# Load the TensorFlow model
saved_model_dir = 'model_tf'

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

# Save the TFLite model
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)


c:\Users\Faturrahman\Downloads\Bahan_Skripsi\yolov5\models\yolo.py:242: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if augment:
c:\Users\Faturrahman\Downloads\Bahan_Skripsi\yolov5\models\yolo.py:153: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if profile:
c:\Users\Faturrahman\Downloads\Bahan_Skripsi\yolov5\models\yolo.py:157: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace

ModuleNotFoundError: No module named 'onnx_tf'

In [1]:
import tf2onnx
import onnx
import tensorflow as tf

# Load the ONNX model
onnx_model = onnx.load("model.onnx")

# Convert to TensorFlow
spec = (tf.TensorSpec((None, 224, 224, 3), tf.float32, name="input"),)  # Adjust the shape according to your input
output_path = "model.pb"  # Or use .saved_model for SavedModel format
model_proto, external_tensor_storage = tf2onnx.convert.from_onnx(
    onnx_model,
    input_signature=spec,
    opset=13,  # Adjust based on the opset version used for ONNX export
    output_path=output_path
)


DecodeError: Error parsing message with type 'tensorflow.GraphDef'

In [1]:
import onnx
from onnx_tf.backend import prepare
import tensorflow as tf

# Step 1: Convert ONNX model to TensorFlow
def convert_onnx_to_tf(onnx_path, tf_path):
    # Load the ONNX model
    onnx_model = onnx.load(onnx_path)

    # Convert to TensorFlow
    tf_rep = prepare(onnx_model)

    # Export the TensorFlow model
    tf_rep.export_graph(tf_path)

# Step 2: Convert TensorFlow model to TensorFlow Lite
def convert_tf_to_tflite(tf_path, tflite_path):
    # Convert the TensorFlow model to TensorFlow Lite
    converter = tf.lite.TFLiteConverter.from_saved_model(tf_path)
    tflite_model = converter.convert()

    # Save the TFLite model
    with open(tflite_path, "wb") as f_out:
        f_out.write(tflite_model)

# Paths for the models
onnx_model_path = "model.onnx"  # Change this to the path of your ONNX model
tf_model_path = "./model.tf"  # Change this to where you want to save the TensorFlow model
tflite_model_path = "model.tflite"  # Change this to where you want to save the TFLite model

# Conversion
convert_onnx_to_tf(onnx_model_path, tf_model_path)
convert_tf_to_tflite(tf_model_path, tflite_model_path)

print("Conversion complete. The TFLite model is saved at:", tflite_model_path)


C:\Users\Faturrahman\AppData\Roaming\Python\Python310\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
C:\Users\Faturrahman\AppData\Roaming\Python\Python310\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug,

INFO:tensorflow:Assets written to: ./model.tf\assets


INFO:tensorflow:Assets written to: ./model.tf\assets


Conversion complete. The TFLite model is saved at: model.tflite
